# 実験結果の可視化

In [1]:
%load_ext autoreload
%autoreload 2

import os
import configparser
import sys

import numpy as np
import pandas as pd
# import torch
import plotly.graph_objects as go

# 設定の読み込み
config = configparser.ConfigParser()
config_path = "../config.ini"
config.read(config_path)
PROJECT_DIR = config["paths"]["project_dir"]
LOG_DIR = config["paths"]["logs_dir"]
sys.path.append(PROJECT_DIR)

from src.utils_experiment import OptimLogParser_v1
from src.utils_experiment import OptimLogParser
from src.utils_experiment import search_log_files

# 08-30, 08-31

In [11]:
log_files = search_log_files(LOG_DIR, ["2024-08-30", "2024-08-31"], logic="or")
log_files

['2024-08-23_2024-08-30',
 '2024-08-30_21-00-46_Simple_5d_unconstrained_ReLU.log',
 '2024-08-30_21-02-48_Simple_5d_unconstrained_tanh.log',
 '2024-08-31_00-17-57_Simple_5d_constrained3_ReLU.log',
 '2024-08-31_00-18-32_Simple_5d_unconstrained_tanh.log',
 '2024-08-31_00-35-08_Simple_5d_constrained3_ReLU_2.log',
 '2024-08-31_00-35-08_Simple_5d_constrained3_ReLU_3.log',
 '2024-08-31_02-40-45_Simple_5d_constrained3_tanh_2.log',
 '2024-08-31_02-40-45_Simple_5d_constrained3_tanh_3.log']

### visualization for constrained bo

In [7]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from src.utils_experiment import OptimLogParser


# Function to parse the log file and return the merged DataFrame
def parse_and_prepare_data(log_file_path):
    parser = OptimLogParser(log_file_path)
    parser.parse_log_file()

    settings = parser.settings
    initial_data = parser.initial_data
    bo_data = parser.bo_data

    columns = [
        "iteration",
        "iteration_time",
        "candidate",
        "function_value",
        "best_f",
        "acquisition_value",
        "surrogate_mean",
        "surrogate_covariance",
        "final_training_loss",
    ]

    df_merged = pd.concat([initial_data, bo_data], axis=0)
    df_merged["best_f"] = df_merged["function_value"].cummax()
    df_merged = df_merged[columns]
    df_merged["iteration"] = df_merged["iteration"].fillna(0).astype(int)

    # Apply constraints
    df_merged["candidate"] = df_merged["candidate"].apply(restore_array)
    df_merged["constraint"] = df_merged["candidate"].apply(g)

    # Calculate best_f_constrained, considering only rows where constraint is True
    df_merged["best_f_constrained"] = df_merged.apply(
        lambda row: row["function_value"] if row["constraint"] else float("-inf"),
        axis=1,
    ).cummax()

    # Replace negative infinity with NaN for rows where constraint is False
    df_merged["best_f_constrained"] = df_merged["best_f_constrained"].replace(
        float("-inf"), np.nan
    )

    return df_merged, settings


# Function to restore array from string representation
def restore_array(str_array):
    str_array = str_array.strip("'").strip("[]")
    array = np.fromstring(str_array, sep=" ")
    return array.reshape(1, -1)


# Constraint function
def g(X):
    """
    Constraint: x1 == x2
    """
    X1 = X[:, 0]
    X2 = X[:, 1]

    return (X1 == X2)[0]


# Function to plot the data
def plot_function_values(df_merged):
    fig = go.Figure()

    # Add scatter trace for function_value with color based on constraint
    fig.add_trace(
        go.Scatter(
            x=df_merged["iteration"],
            y=df_merged["function_value"],
            mode="markers",
            marker=dict(
                color=df_merged["constraint"].apply(lambda x: "orange" if x else "blue")
            ),
            name="Function Value",
        )
    )

    # Add line trace for best_f
    fig.add_trace(
        go.Scatter(
            x=df_merged["iteration"],
            y=df_merged["best_f"],
            mode="lines",
            line=dict(color="red"),
            name="Best f",
        )
    )

    # Add line trace for best_f_constrained
    fig.add_trace(
        go.Scatter(
            x=df_merged["iteration"],
            y=df_merged["best_f_constrained"],
            mode="lines",
            line=dict(color="green"),
            name="Best f under constraint",
        )
    )

    # Update layout
    fig.update_layout(
        title="Function Value over Iterations with Constraints",
        xaxis_title="Iteration",
        yaxis_title="Function Value",
        showlegend=True,
    )

    # Show the figure
    fig.show()


# # Example usage ------------------------------------------------------------------------------
# log_file_path = os.path.join(
#     LOG_DIR, "2024-08-27_20-12-18_Simple_5d_constrained_normalized-input_2.log"
# )
# df_merged, settings = parse_and_prepare_data(log_file_path)

# print("実験設定")
# display(settings)
# print()
# display(df_merged)

# plot_function_values(df_merged)


In [20]:
filename = log_files[1]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-30_21-00-46_Simple_5d_unconstrained_ReLU.log

実験設定


{'name': 'Simple_5d_unconstrained_ReLU',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': ReLU()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'ReLU activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[-7.0, -10.0, 2.0, 8.0, 4.0]]",-233.0,-233.0,NaN,NaN,NaN,233.493118,False,NaN
0,1,0.0771,"[[-4.0, -7.0, 5.0, -6.0, 7.0]]",-175.0,-175.0,31.216619,-0.804090,119.027054,17.365740,False,NaN
1,2,0.0648,"[[-0.0, -2.0, -2.0, -3.0, -5.0]]",-42.0,-42.0,-1.090824,-38.773975,1372.182617,8.348160,False,NaN
2,3,0.0722,"[[-4.0, -4.0, -3.0, -6.0, -6.0]]",-113.0,-42.0,-32.147949,-73.220337,5216.841797,5.806534,True,-113.0
3,4,0.0651,"[[-2.0, -5.0, -5.0, -3.0, -6.0]]",-99.0,-42.0,-3.486019,69.974205,7444.909668,7.361945,False,-113.0


In [28]:
filename = log_files[2]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-30_21-02-48_Simple_5d_unconstrained_tanh.log

実験設定


{'name': 'Simple_5d_unconstrained_tanh',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': Tanh()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'tanh activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[1.0, -3.0, 6.0, -2.0, -6.0]]",-86.0,-86.0,NaN,NaN,NaN,67.108185,False,NaN
0,1,0.1096,"[[-6.0, -6.0, -5.0, -1.0, -6.0]]",-134.0,-86.0,13.070188,-5.468557,53.569790,53.895958,True,-134.0
1,2,0.1051,"[[-0.0, -1.0, -3.0, -3.0, -5.0]]",-44.0,-44.0,11.514765,-25.871372,84.454407,35.807270,False,-134.0
2,3,0.1119,"[[-4.0, -7.0, 8.0, -6.0, 0.0]]",-165.0,-44.0,2.458394,-9.547071,97.995094,53.072948,False,-134.0
3,4,0.1089,"[[3.0, 1.0, -8.0, -5.0, -4.0]]",-115.0,-44.0,25.794195,-42.476017,114.620201,22.017035,False,-134.0


In [29]:
filename = log_files[3]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-31_00-17-57_Simple_5d_constrained3_ReLU.log

実験設定


{'name': 'Simple_5d_constrained3_ReLU',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': ReLU()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001, 'lambda1': 0.5},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'ReLU activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[-5.0, 6.0, 5.0, 10.0, 9.0]]",-267.0,-267.0,NaN,NaN,NaN,0.242262,False,NaN
0,1,0.1273,"[[-8.0, -9.0, 10.0, -4.0, 9.0]]",-342.0,-267.0,-10.132787,-1.497589,1.448306,0.875610,False,NaN
1,2,0.1260,"[[-2.0, 5.0, -2.0, -7.0, -5.0]]",-107.0,-107.0,-2.272626,-0.874946,2.278536,-4.915788,False,NaN
2,3,0.1257,"[[-6.0, -6.0, -2.0, -4.0, -3.0]]",-101.0,-101.0,-6.846144,-5.784340,16.507420,-63.153091,True,-101.0
3,4,0.1268,"[[-10.0, -10.0, -10.0, -9.0, -10.0]]",-481.0,-101.0,-27.915590,-47.738392,451.132568,-111.485863,True,-101.0


In [30]:
df_merged

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[-5.0, 6.0, 5.0, 10.0, 9.0]]",-267.0,-267.0,NaN,NaN,NaN,2.422620e-01,False,NaN
0,1,0.1273,"[[-8.0, -9.0, 10.0, -4.0, 9.0]]",-342.0,-267.0,-1.013279e+01,-1.497589e+00,1.448306,8.756100e-01,False,NaN
1,2,0.1260,"[[-2.0, 5.0, -2.0, -7.0, -5.0]]",-107.0,-107.0,-2.272626e+00,-8.749464e-01,2.278536,-4.915788e+00,False,NaN
2,3,0.1257,"[[-6.0, -6.0, -2.0, -4.0, -3.0]]",-101.0,-101.0,-6.846144e+00,-5.784340e+00,16.507420,-6.315309e+01,True,-101.0
3,4,0.1268,"[[-10.0, -10.0, -10.0, -9.0, -10.0]]",-481.0,-101.0,-2.791559e+01,-4.773839e+01,451.132568,-1.114859e+02,True,-101.0
...,...,...,...,...,...,...,...,...,...,...,...
995,996,1.1329,"[[-10.0, -10.0, -10.0, -10.0, -10.0]]",-500.0,-33.0,-2.350658e+08,-3.483620e+12,1.491326,-8.799242e+11,True,-49.0
996,997,1.0482,"[[-10.0, -10.0, -10.0, -10.0, -10.0]]",-500.0,-33.0,-2.364979e+08,-3.503901e+12,1.492654,-8.846126e+11,True,-49.0
997,998,1.0422,"[[-10.0, -10.0, -10.0, -10.0, -10.0]]",-500.0,-33.0,-2.377883e+08,-3.510873e+12,1.502377,-8.849113e+11,True,-49.0
998,999,1.0999,"[[-10.0, -10.0, -10.0, -10.0, -10.0]]",-500.0,-33.0,-2.390209e+08,-3.532809e+12,1.535984,-8.868764e+11,True,-49.0


In [31]:
filename = log_files[4]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-31_00-18-32_Simple_5d_unconstrained_tanh.log

実験設定


{'name': 'Simple_5d_unconstrained_tanh',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': Tanh()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'Tanh activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[-9.0, 7.0, 10.0, 1.0, 4.0]]",-247.0,-247.0,NaN,NaN,NaN,495.501007,False,NaN
0,1,0.0775,"[[-5.0, -8.0, -2.0, -2.0, -4.0]]",-113.0,-113.0,7.484067,-11.204698,52.620525,167.885971,False,NaN
1,2,0.0862,"[[-0.0, -5.0, -0.0, -5.0, -6.0]]",-86.0,-86.0,26.213003,-32.562824,86.640724,110.422249,False,NaN
2,3,0.0816,"[[-10.0, -8.0, -0.0, -6.0, -3.0]]",-209.0,-86.0,38.483929,-5.026519,103.532143,108.087646,False,NaN
3,4,0.0725,"[[7.0, -4.0, -7.0, -8.0, -7.0]]",-227.0,-86.0,18.319443,-31.230209,123.198151,98.939415,False,NaN


In [32]:
filename = log_files[5]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-31_00-35-08_Simple_5d_constrained3_ReLU_2.log

実験設定


{'name': 'Simple_5d_constrained3_ReLU_2',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': ReLU()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001, 'lambda1': 0.2},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'ReLU activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[-2.0, -1.0, -10.0, 3.0, -7.0]]",-163.0,-163.0,NaN,NaN,NaN,-0.245632,False,NaN
0,1,0.1659,"[[7.0, 1.0, -6.0, 6.0, -1.0]]",-123.0,-123.0,-6.969411,-1.724248,1.490017,-1.366055,False,NaN
1,2,0.1702,"[[-1.0, -2.0, -7.0, 8.0, 8.0]]",-182.0,-123.0,-12.764651,-3.252240,4.737168,-3.153183,False,NaN
2,3,0.1846,"[[-0.0, -1.0, 1.0, -2.0, -1.0]]",-7.0,-7.0,-1.221062,-15.007815,38.205597,-25.245432,False,NaN
3,4,0.1676,"[[-1.0, -8.0, -5.0, -4.0, -7.0]]",-155.0,-7.0,-18.514225,-37.704521,551.022644,-62.874763,False,NaN


In [33]:
filename = log_files[6]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-31_00-35-08_Simple_5d_constrained3_ReLU_3.log

実験設定


{'name': 'Simple_5d_constrained3_ReLU_3',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': ReLU()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001, 'lambda1': 0},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'ReLU activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[9.0, -10.0, 1.0, -8.0, -4.0]]",-262.0,-262.0,NaN,NaN,NaN,0.0,False,NaN
0,1,0.1756,"[[7.0, 2.0, -10.0, 1.0, -3.0]]",-163.0,-163.0,-3.560579,1.765620,1.268620,0.0,False,NaN
1,2,0.1769,"[[8.0, -3.0, -7.0, -7.0, -3.0]]",-180.0,-163.0,-3.892407,-1.922864,1.258985,0.0,False,NaN
2,3,0.1762,"[[3.0, 4.0, -4.0, 3.0, -6.0]]",-86.0,-86.0,-0.850657,1.288904,1.286020,0.0,False,NaN
3,4,0.1641,"[[-1.0, -8.0, -6.0, -6.0, 6.0]]",-173.0,-86.0,-0.106960,-1.477705,1.234640,0.0,False,NaN


In [34]:
filename = log_files[7]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-31_02-40-45_Simple_5d_constrained3_tanh_2.log

実験設定


{'name': 'Simple_5d_constrained3_tanh_2',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': Tanh()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001, 'lambda1': 0.2},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'tanh activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[1.0, -2.0, 6.0, 1.0, -1.0]]",-43.0,-43.0,NaN,NaN,NaN,0.097791,False,NaN
0,1,0.1608,"[[-2.0, 4.0, -6.0, -3.0, 2.0]]",-69.0,-43.0,1.364601,-3.313272,5.288722,-2.325277,False,NaN
1,2,0.1388,"[[3.0, -0.0, -10.0, -2.0, -4.0]]",-129.0,-43.0,7.487433,-5.159888,21.078367,0.664669,False,NaN
2,3,0.1382,"[[-5.0, -7.0, -8.0, -6.0, -7.0]]",-223.0,-43.0,15.986557,-14.448198,50.017021,-3.322469,False,NaN
3,4,0.1605,"[[-2.0, -3.0, -7.0, -8.0, -5.0]]",-151.0,-43.0,27.067118,-16.305901,61.292427,-4.593727,False,NaN


In [35]:
filename = log_files[8]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-31_02-40-45_Simple_5d_constrained3_tanh_3.log

実験設定


{'name': 'Simple_5d_constrained3_tanh_3',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': Tanh()},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001, 'lambda1': 0},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'tanh activation function'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[2.0, 8.0, 4.0, -3.0, 9.0]]",-174.0,-174.0,NaN,NaN,NaN,0.0,False,NaN
0,1,0.1570,"[[4.0, 3.0, -6.0, -9.0, -3.0]]",-151.0,-151.0,-4.936556,-2.028985,1.836713,0.0,False,NaN
1,2,0.1555,"[[-1.0, 4.0, 1.0, -4.0, -6.0]]",-70.0,-70.0,2.969326,-1.388929,1.855211,0.0,False,NaN
2,3,0.1408,"[[-9.0, 10.0, -10.0, -8.0, 9.0]]",-426.0,-70.0,-2.646729,-2.834899,1.931112,0.0,False,NaN
3,4,0.1564,"[[-2.0, -4.0, 2.0, -9.0, 6.0]]",-141.0,-70.0,5.548716,0.408800,1.988399,0.0,False,NaN


# clipping

In [45]:
log_files = search_log_files(LOG_DIR, ["2024-08-30", "2024-08-31"], logic="or")
log_files = log_files[9:]
log_files

['2024-08-31_03-43-20_Simple_5d_constrained3_ReLU.log',
 '2024-08-31_03-43-20_Simple_5d_constrained3_ReLU_2.log',
 '2024-08-31_03-43-20_Simple_5d_constrained3_ReLU_3.log',
 '2024-08-31_03-43-20_Simple_5d_constrained3_tanh.log',
 '2024-08-31_03-43-20_Simple_5d_constrained3_tanh_2.log',
 '2024-08-31_03-43-20_Simple_5d_constrained3_tanh_3.log']

In [46]:
filename = log_files[0]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-31_03-43-20_Simple_5d_constrained3_ReLU.log

実験設定


{'name': 'Simple_5d_constrained3_ReLU',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': ReLU(),
  'min_val': -500,
  'max_val': 500},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001, 'lambda1': 0.5},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'try output clipping of surrogate model'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[7.0, 8.0, 5.0, 7.0, -5.0]]",-212.0,-212.0,NaN,NaN,NaN,0.664230,False,NaN
0,1,0.2880,"[[-6.0, -9.0, 3.0, -5.0, -7.0]]",-200.0,-200.0,8.614080,-0.453802,1.342285,2.584203,False,NaN
1,2,0.3812,"[[-4.0, -5.0, -7.0, -1.0, 2.0]]",-95.0,-95.0,-0.335804,-3.850627,4.170219,-11.544804,False,NaN
2,3,0.2902,"[[-5.0, 1.0, -4.0, -8.0, -9.0]]",-187.0,-95.0,-13.307566,-6.421475,37.170166,-33.281513,False,NaN
3,4,0.3021,"[[-3.0, -4.0, -3.0, -5.0, -2.0]]",-63.0,-63.0,-3.094020,-120.276176,865.037598,-72.874344,False,NaN


In [48]:
filename = log_files[1]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-31_03-43-20_Simple_5d_constrained3_ReLU_2.log

実験設定


{'name': 'Simple_5d_constrained3_ReLU_2',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': ReLU(),
  'min_val': -500,
  'max_val': 500},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001, 'lambda1': 0.2},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'try output clipping of surrogate model'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[-10.0, -8.0, 0.0, -2.0, -5.0]]",-193.0,-193.0,NaN,NaN,NaN,0.064959,False,NaN
0,1,0.3026,"[[4.0, -3.0, 9.0, -1.0, -9.0]]",-188.0,-188.0,2.729732,-1.872662,1.814041,-2.214407,False,NaN
1,2,0.3741,"[[8.0, 0.0, 3.0, 1.0, -6.0]]",-110.0,-110.0,-5.104043,1.211333,8.688253,-3.091572,False,NaN
2,3,0.3396,"[[-3.0, -1.0, -2.0, -4.0, -1.0]]",-31.0,-31.0,-3.679221,-11.311790,62.285698,-25.117334,False,NaN
3,4,0.2991,"[[-5.0, -5.0, -4.0, -7.0, -3.0]]",-124.0,-31.0,-4.347231,-80.212624,751.396667,-32.782368,True,-124.0


In [49]:
filename = log_files[2]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-31_03-43-20_Simple_5d_constrained3_ReLU_3.log

実験設定


{'name': 'Simple_5d_constrained3_ReLU_3',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': ReLU(),
  'min_val': -500,
  'max_val': 500},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001, 'lambda1': 0},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'try output clipping of surrogate model'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[8.0, -10.0, 5.0, -4.0, 3.0]]",-214.0,-214.0,NaN,NaN,NaN,0.0,False,NaN
0,1,0.3263,"[[7.0, -7.0, -10.0, 3.0, -9.0]]",-288.0,-214.0,-1.979694,0.764574,1.294533,0.0,False,NaN
1,2,0.4036,"[[5.0, -5.0, 8.0, 4.0, 8.0]]",-194.0,-194.0,5.362859,0.854817,1.410656,0.0,False,NaN
2,3,0.3525,"[[-1.0, -9.0, 7.0, -10.0, 7.0]]",-280.0,-194.0,-2.247616,-1.057850,1.383889,0.0,False,NaN
3,4,0.3647,"[[10.0, -3.0, -4.0, 3.0, 8.0]]",-198.0,-194.0,2.761843,-0.294501,1.348395,0.0,False,NaN


In [50]:
filename = log_files[3]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-31_03-43-20_Simple_5d_constrained3_tanh.log

実験設定


{'name': 'Simple_5d_constrained3_tanh',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': Tanh(),
  'min_val': -500,
  'max_val': 500},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001, 'lambda1': 0.5},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'try output clipping of surrogate model'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[7.0, 10.0, -9.0, 8.0, 4.0]]",-310.0,-310.0,NaN,NaN,NaN,3.308631,False,NaN
0,1,0.2980,"[[-5.0, -7.0, -2.0, 6.0, 6.0]]",-150.0,-150.0,-1.262798,2.489374,6.911854,-7.099444,False,NaN
1,2,0.3919,"[[-10.0, -10.0, 5.0, -2.0, -3.0]]",-238.0,-150.0,8.539700,-19.899693,25.552040,147.435501,True,-238.0
2,3,0.2794,"[[-5.0, 3.0, -9.0, -10.0, -1.0]]",-216.0,-150.0,18.608723,-19.694468,79.540085,115.647278,False,-238.0
3,4,0.2889,"[[-0.0, -9.0, 1.0, -10.0, -2.0]]",-186.0,-150.0,26.051754,-51.322788,96.366753,94.763634,False,-238.0


In [51]:
filename = log_files[4]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-31_03-43-20_Simple_5d_constrained3_tanh_2.log

実験設定


{'name': 'Simple_5d_constrained3_tanh_2',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': Tanh(),
  'min_val': -500,
  'max_val': 500},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001, 'lambda1': 0.2},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'try output clipping of surrogate model'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[-3.0, -7.0, -5.0, 9.0, -4.0]]",-180.0,-180.0,NaN,NaN,NaN,-0.633043,False,NaN
0,1,0.2794,"[[-2.0, -8.0, -5.0, -9.0, -9.0]]",-255.0,-180.0,1.448561,-3.145479,6.302875,0.132680,False,NaN
1,2,0.3998,"[[-8.0, 2.0, -7.0, -8.0, -2.0]]",-185.0,-180.0,2.377535,-6.589172,27.708204,-1.780574,False,NaN
2,3,0.3199,"[[-4.0, -4.0, -8.0, -5.0, -5.0]]",-146.0,-146.0,7.589122,-8.711795,53.039497,159.208252,True,-146.0
3,4,0.3005,"[[-2.0, -1.0, -9.0, -1.0, -2.0]]",-91.0,-91.0,15.893245,-27.993193,76.988953,99.011192,False,-146.0


In [52]:
filename = log_files[5]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-31_03-43-20_Simple_5d_constrained3_tanh_3.log

実験設定


{'name': 'Simple_5d_constrained3_tanh_3',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': Tanh(),
  'min_val': -500,
  'max_val': 500},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001, 'lambda1': 0},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'try output clipping of surrogate model'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[-1.0, 2.0, -4.0, -1.0, -2.0]]",-26.0,-26.0,NaN,NaN,NaN,0.0,False,NaN
0,1,0.2994,"[[1.0, -9.0, -2.0, -4.0, 7.0]]",-151.0,-26.0,1.141919,-0.970434,1.872572,0.0,False,NaN
1,2,0.3869,"[[9.0, -8.0, -6.0, 4.0, 7.0]]",-246.0,-26.0,1.798641,-1.363256,1.804495,0.0,False,NaN
2,3,0.2637,"[[-0.0, 4.0, -1.0, 3.0, -5.0]]",-51.0,-26.0,1.230301,0.979199,1.954132,0.0,False,NaN
3,4,0.2845,"[[-9.0, 2.0, 3.0, 9.0, 2.0]]",-179.0,-26.0,2.147081,1.292902,2.319996,0.0,False,NaN


In [53]:
log_files = search_log_files(LOG_DIR, ["2024-08-30", "2024-08-31"], logic="or")
log_files = log_files[9:]
log_files

['2024-08-31_03-43-20_Simple_5d_constrained3_ReLU.log',
 '2024-08-31_03-43-20_Simple_5d_constrained3_ReLU_2.log',
 '2024-08-31_03-43-20_Simple_5d_constrained3_ReLU_3.log',
 '2024-08-31_03-43-20_Simple_5d_constrained3_tanh.log',
 '2024-08-31_03-43-20_Simple_5d_constrained3_tanh_2.log',
 '2024-08-31_03-43-20_Simple_5d_constrained3_tanh_3.log',
 '2024-08-31_04-37-36_Simple_5d_constrained3_LeakyReLU.log',
 '2024-08-31_04-37-36_Simple_5d_constrained3_LeakyReLU_2.log',
 '2024-08-31_04-37-36_Simple_5d_constrained3_LeakyReLU_3.log']

In [55]:
filename = log_files[6]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-31_04-37-36_Simple_5d_constrained3_LeakyReLU.log

実験設定


{'name': 'Simple_5d_constrained3_LeakyReLU',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': LeakyReLU(negative_slope=0.01),
  'min_val': -500,
  'max_val': 500},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001, 'lambda1': 0.5},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'try output clipping of surrogate model'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[2.0, -4.0, -7.0, 9.0, -2.0]]",-154.0,-154.0,NaN,NaN,NaN,1.170707,False,NaN
0,1,0.1697,"[[-0.0, -6.0, 2.0, 7.0, -7.0]]",-138.0,-138.0,4.876683,-0.759485,1.611076,-0.521493,False,NaN
1,2,0.1593,"[[-2.0, -10.0, 9.0, -5.0, -9.0]]",-291.0,-138.0,15.502991,-2.724863,2.702632,-4.046331,False,NaN
2,3,0.1733,"[[-4.0, -10.0, -10.0, -5.0, -9.0]]",-322.0,-138.0,119.623894,-0.288764,19.130501,0.272754,False,NaN
3,4,0.2455,"[[-10.0, -10.0, -10.0, -10.0, -10.0]]",-500.0,-138.0,586.989136,-28.788601,169.848816,-153.632034,True,-500.0


In [56]:
filename = log_files[7]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-31_04-37-36_Simple_5d_constrained3_LeakyReLU_2.log

実験設定


{'name': 'Simple_5d_constrained3_LeakyReLU_2',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': LeakyReLU(negative_slope=0.01),
  'min_val': -500,
  'max_val': 500},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001, 'lambda1': 0.2},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'try output clipping of surrogate model'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[-5.0, -6.0, -3.0, -7.0, 2.0]]",-123.0,-123.0,NaN,NaN,NaN,0.052986,False,NaN
0,1,0.1698,"[[6.0, 2.0, 7.0, 5.0, 6.0]]",-150.0,-123.0,3.141046,0.149287,1.452963,-0.392505,False,NaN
1,2,0.1625,"[[9.0, 0.0, -3.0, 8.0, -2.0]]",-158.0,-123.0,-3.167199,0.611614,4.582361,-2.069824,False,NaN
2,3,0.1690,"[[-5.0, -10.0, 9.0, -10.0, 3.0]]",-315.0,-123.0,101.587341,-4.865533,16.465584,-8.133229,False,NaN
3,4,0.2392,"[[-10.0, -10.0, -10.0, -10.0, -10.0]]",-500.0,-123.0,609.186096,-16.383039,106.635384,-7.660076,True,-500.0


In [57]:
filename = log_files[8]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-31_04-37-36_Simple_5d_constrained3_LeakyReLU_3.log

実験設定


{'name': 'Simple_5d_constrained3_LeakyReLU_3',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': LeakyReLU(negative_slope=0.01),
  'min_val': -500,
  'max_val': 500},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001, 'lambda1': 0},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'try output clipping of surrogate model'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[10.0, -9.0, 10.0, 0.0, 0.0]]",-281.0,-281.0,NaN,NaN,NaN,0.0,False,NaN
0,1,0.1744,"[[-3.0, 4.0, -1.0, -7.0, -8.0]]",-139.0,-139.0,1.282300,-1.160049,1.469270,0.0,False,NaN
1,2,0.1626,"[[-0.0, 2.0, 8.0, 5.0, -7.0]]",-142.0,-139.0,-3.773517,1.132739,1.701113,0.0,False,NaN
2,3,0.1690,"[[4.0, -8.0, 5.0, -2.0, -1.0]]",-110.0,-110.0,2.675642,0.037421,1.603449,0.0,False,NaN
3,4,0.2349,"[[-8.0, -6.0, 8.0, 6.0, -10.0]]",-300.0,-110.0,4.359490,0.665133,1.666791,0.0,False,NaN


# Discrete

In [5]:
log_files = search_log_files(LOG_DIR, ["2024-08-31"])
log_files

['2024-08-31_00-17-57_Simple_5d_constrained3_ReLU.log',
 '2024-08-31_00-18-32_Simple_5d_unconstrained_tanh.log',
 '2024-08-31_00-35-08_Simple_5d_constrained3_ReLU_2.log',
 '2024-08-31_00-35-08_Simple_5d_constrained3_ReLU_3.log',
 '2024-08-31_02-40-45_Simple_5d_constrained3_tanh_2.log',
 '2024-08-31_02-40-45_Simple_5d_constrained3_tanh_3.log',
 '2024-08-31_03-43-20_Simple_5d_constrained3_ReLU.log',
 '2024-08-31_03-43-20_Simple_5d_constrained3_ReLU_2.log',
 '2024-08-31_03-43-20_Simple_5d_constrained3_ReLU_3.log',
 '2024-08-31_03-43-20_Simple_5d_constrained3_tanh.log',
 '2024-08-31_03-43-20_Simple_5d_constrained3_tanh_2.log',
 '2024-08-31_03-43-20_Simple_5d_constrained3_tanh_3.log',
 '2024-08-31_04-37-36_Simple_5d_constrained3_LeakyReLU.log',
 '2024-08-31_04-37-36_Simple_5d_constrained3_LeakyReLU_2.log',
 '2024-08-31_04-37-36_Simple_5d_constrained3_LeakyReLU_3.log',
 '2024-08-31_12-48-37_Simple_5d_unconstrained_tanh.log']

In [8]:
filename = log_files[-1]
print(filename)
print()

log_file_path = os.path.join(LOG_DIR, filename)
df_merged, settings = parse_and_prepare_data(log_file_path)

print("実験設定")
display(settings)
print()
display(df_merged.head())

plot_function_values(df_merged)

2024-08-31_12-48-37_Simple_5d_unconstrained_tanh.log

実験設定


{'name': 'Simple_5d_unconstrained_tanh',
 'device': 'cpu',
 'bo_iter': 1000,
 'initial_data_size': 1,
 'model': {'hidden_unit_size': 64,
  'num_hidden_layers': 3,
  'activation_fn': Tanh(),
  'min_val': None,
  'max_val': None},
 'model_optim': {'num_epochs': 100, 'learning_rate': 0.001},
 'acquisition_optim': {'beta': 0.1, 'num_restarts': 5, 'raw_samples': 20},
 'memo': 'try output clipping of surrogate model'}

,iteration,iteration_time,candidate,function_value,best_f,acquisition_value,surrogate_mean,surrogate_covariance,final_training_loss,constraint,best_f_constrained
0,0,NaN,"[[-5.0, -7.0, -7.0, 5.0, 8.0]]",-212.0,-212.0,NaN,NaN,NaN,375.952942,False,NaN
0,1,0.0661,"[[-8.0, -4.0, -8.0, -8.0, 4.0]]",-224.0,-212.0,2.941620,-6.686142,53.308273,251.043701,False,NaN
1,2,0.0669,"[[-0.0, -10.0, 0.0, -8.0, -5.0]]",-189.0,-189.0,28.131876,-10.304266,86.104729,182.087326,False,NaN
2,3,0.0681,"[[-4.0, -4.0, -2.0, -3.0, -4.0]]",-61.0,-61.0,24.715063,2.533624,103.693848,136.031723,True,-61.0
3,4,0.0637,"[[-8.0, -2.0, -10.0, -2.0, -4.0]]",-188.0,-61.0,34.725651,-16.773941,123.462189,98.806564,False,-61.0
